In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [54]:
url = "https://app.metricwire.com/"
my_email = '***'
my_passw = '***'

In [16]:
date = '27-10-2020'

In [92]:
driver = webdriver.Chrome()
driver.get(url)

time.sleep(1)
email_inp = driver.find_element_by_css_selector("#root > div > div > div > div > div > form > div:nth-child(4) > div > input")
email_inp.send_keys(my_email)

pass_inp = driver.find_element_by_css_selector("#root > div > div > div > div > div > form > div:nth-child(5) > div > input")
pass_inp.send_keys(my_passw)

login_btn = driver.find_element_by_css_selector("#root > div > div > div > div > div > form > div.card-action.border-0.text-right.mt-3 > button > span.MuiButton-label")
login_btn.click()

time.sleep(3)
sbdl_wksp = driver.find_element_by_css_selector("#root > div > main > div > div > div.MuiGrid-root.MuiGrid-item.MuiGrid-grid-xs-12.MuiGrid-grid-sm-12.MuiGrid-grid-md-8 > div > div.MuiCardContent-root > table > tbody > tr:nth-child(2) > td:nth-child(3) > a > span.MuiButton-label")
sbdl_wksp.click()

time.sleep(1)
sbdl_study = driver.find_element_by_css_selector("#root > div > main > div > div.MuiGrid-root.MuiGrid-container.MuiGrid-spacing-xs-4 > div:nth-child(2) > a > div > div > div.MuiGrid-root.MuiGrid-item.MuiGrid-grid-xs-8")
sbdl_study.click()

time.sleep(1)
sbdl_view = driver.find_element_by_css_selector("#root > div > main > div > div > div > div > table > tbody > tr > td:nth-child(4) > a > button > span.MuiButton-label")
sbdl_view.click()

time.sleep(5)
part_btn = driver.find_element_by_css_selector("#root > div > main > div > div > header > div > div > div > div > div > button:nth-child(6)")
part_btn.click()

time.sleep(7)
sort_btn = driver.find_element_by_css_selector("#root > div > main > div > div > div > div > div.MuiPaper-root.MuiCard-root.MuiPaper-elevation1.MuiPaper-rounded > div > div > div > div.jss97 > div > div > div > table > thead > tr > th:nth-child(10) > span")
sort_btn.click()

time.sleep(3)
expand_btn = driver.find_element_by_xpath("/html/body/div[1]/div/main/div/div/div/div/div[2]/div/div/div/table/tfoot/tr/td/div/div[2]/div")
expand_btn.click()

sel300 = driver.find_element_by_xpath("/html/body/div[4]/div[3]/ul/li[3]")
sel300.click()

In [ ]:
table = driver.find_element_by_css_selector('#root > div > main > div > div > div > div > div.MuiPaper-root.MuiCard-root.MuiPaper-elevation1.MuiPaper-rounded > div > div > div > div.jss97 > div > div > div > table > tbody')

#iterate through participants
row_num = 1
participant_info = []
for row in table.find_elements_by_css_selector('tr'):
    row_num +=1 
    by_part = []
    enroll_date = driver.find_element_by_css_selector('#root > div > main > div > div > div > div > div.MuiPaper-root.MuiCard-root.MuiPaper-elevation1.MuiPaper-rounded > div > div > div > div.jss97 > div > div > div > table > tbody > tr:nth-child(' + str(row_num) + ') > td:nth-child(10)')
 
    if enroll_date.text[:10] == date:
        email_ob = driver.find_element_by_css_selector('#root > div > main > div > div > div > div > div.MuiPaper-root.MuiCard-root.MuiPaper-elevation1.MuiPaper-rounded > div > div > div > div.jss97 > div > div > div > table > tbody > tr:nth-child(' + str(row_num) + ') > td:nth-child(5)')
        email = email_ob.text.split('@')[0]
        print(email)
        by_part.append(email)
        
        launch_btn = driver.find_element_by_css_selector('#root > div > main > div > div > div > div > div.MuiPaper-root.MuiCard-root.MuiPaper-elevation1.MuiPaper-rounded > div > div > div > div.jss97 > div > div > div > table > tbody > tr:nth-child(' + str(row_num) + ') > td:nth-child(7) > a')
        participant_link = launch_btn.get_attribute('href')
        
        driver.execute_script("window.open('{}');".format(participant_link))
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(5
        fullsched_btn = driver.find_element_by_css_selector('#root > div > main > div > div > div.MuiGrid-root.MuiGrid-item.MuiGrid-grid-xs-8 > div > div > div > div > div > div > div:nth-child(1) > div > div:nth-child(2) > div > button')
        fullsched_btn.click()
        time.sleep(2)
        submissions = driver.find_element_by_css_selector('#root > div > main > div > div > div.MuiGrid-root.MuiGrid-item.MuiGrid-grid-xs-8 > div > div > div > div > div > div > div.MuiCardContent-root.text-xs-center > div > div:nth-child(4) > h3').text
        print(submissions)
        by_part.append(submissions)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    participant_info.append(by_part)
        
       